## Data preparation swissgeol.ch and map.geo.admin.ch

# Borehole data

#### Editor: ON; Date: 10.11.2020
#### Modified: 13.6.2021

# ----------- Let the Script begin ... ----------------

# Importing Libraries

In [1]:
# Import libraries
import os
import datetime
import pandas as pd
import numpy as np
import glob
import geopandas as gpd
from shapely.geometry import Point
import matplotlib as mpl 
import matplotlib.pyplot as plt

# Import 'little helpers' for convenience
import onUtils

# for opening filedialog
import tkinter as tk
from tkinter import filedialog

import json

# For Pandas
#pd.set_option('max_columns', None)
#pd.set_option('max_rows', None)

# Loading Configuration

In [2]:
# Load Configuration
with open("/Users/oesterli/Desktop/python_space/bh_prep/data/input/helper/config.json", "r") as jsonfile:
    conf = json.load(jsonfile)
    print("Coniguration read successfully")


Coniguration read successfully


# Variables

In [3]:
# Define variables

## timestamp for out_dir
now = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

## Convert number of run (run_num) to two digit str
#run_dir = '_'.join(['run',now,str(run_num).zfill(2)])
run_dir = '_'.join(['RUN',now])

## Define output directory path
out_dir_run = os.path.join(conf["root_dir"],'data/output',run_dir)

# Define name of log-file
log_name = '_'.join(['LOG',now])

## input directory for borhole data
in_dir_bh = os.path.join(conf["root_dir"], 'data/input/bh')

## input directory for helper files
in_dir_hp = os.path.join(conf["root_dir"],'data/input/helper')

## Define valide date range
startD = '1700-01-01'
endD = '2200-01-01'


# Functions

### Create run directory

In [4]:
def make_run_dir(f_out_dir_run):
    '''
    Check if run directory already exists. If no, create directory.
    Parameter: run directory path
    '''
    # Check if out_dir already exists
    if os.path.isdir(f_out_dir_run):
        print('ok! Out directory already exists.')
    else:
        print('Output directory not existing, directory will de created')
        os.mkdir(f_out_dir_run)


### Import data exported from GeODin 

In [5]:
def read_input(f_in_dir_bh):
    '''
    Read input exel-files(.xlsx) from input directory. 
    Join all input files into one Pandas Dataframe.
    Parameter: input directory of boreholes
    '''
    
    # create global variable
    global all_data
    
    # Read all .xlsx-Files from source directory
    in_data = glob.glob(os.path.join(f_in_dir_bh,'*.xlsx'))
    
    # Read all data and combine it into one file. Continuos index will be created 
    all_data = pd.DataFrame()

    for f in in_data:
        df = pd.read_excel(f, date_parser=False, dtype={'BOHREDAT':str}) # IMPORTANT: deactive date paser und set BOHREDAT = str
        all_data = all_data.append(df,ignore_index=True)

    

In [ ]:
# Deleting global variables
#del globals()['all_data']

# Inspecting data
### Data types of selected columns

In [6]:
# Inspect dtypes of selected columns
def check_dtypes():
    '''Doc String'''
    #Check dtypes of selected columns
    print("Data types of selected data:\n",all_data[conf["sel_cols_2"]].dtypes)



### Statistics

In [7]:
# Create some simple statistics
def create_stats():
    # Create some simple statistics
    num_bh = len(all_data['SHORTNAME'].unique())
    num_layers = all_data.shape[0]
    print('Number of individual borehole: ', num_bh)
    print('Number of individual layers: ', num_layers)

    # calculate number of layers per borehole
    layer_per_bh = all_data.groupby('SHORTNAME')['DEPTHFROM'].nunique()
    print(layer_per_bh.describe())

# Preprocessing raw data

### Check and correct dates

In [8]:
def correct_dates():
    '''Doc String'''
    # Check individual types of BOHREDAT
    bohredatTypes = all_data['BOHREDAT'].apply(type).unique()

    # Check max, min of dates 
    for i in bohredatTypes:

        print('Max: ', i, all_data['BOHREDAT'].loc[all_data['BOHREDAT'].apply(type) == i].unique().max())
        print('Min: ', i, all_data['BOHREDAT'].loc[all_data['BOHREDAT'].apply(type) == i].unique().min())
    
    # Replace wrong values and values out of bound
    all_data['BOHREDAT_1'] = all_data['BOHREDAT'].apply(lambda x: '1968-06-17' if str(x) == '17.06.1668' else ('1700-01-01' if str(x) < startD else ('2200-01-01' if str(x) > endD else x)))

    # convert column to datetime
    all_data['BOHREDAT_1'] = pd.to_datetime(all_data['BOHREDAT_1'], infer_datetime_format=True, dayfirst=True, errors='raise')

    # overwrite column
    all_data['BOHREDAT'] = all_data['BOHREDAT_1']
    


### Sort data

In [23]:
def sort_data():
    # Sort data by "SHORTNAME" and "DEPTHFROM"
    
    global all_data_sorted
    
    all_data_sorted = all_data.sort_values(by=["SHORTNAME", "DEPTHFROM"])

    # Create an index column
    all_data_sorted['index'] = all_data_sorted.index

    print('Shape of data set: ', all_data_sorted.shape)

### Round columns

In [13]:
def round_data():
    # Round data
    #df['DataFrame column'].round(decimals=number of decimal places needed)
    all_data_sorted['XCOORD'] = all_data_sorted['XCOORD'].round(decimals=2)
    all_data_sorted['YCOORD'] = all_data_sorted['YCOORD'].round(decimals=2)
    all_data_sorted['ZCOORDB'] = all_data_sorted['ZCOORDB'].round(decimals=2)
    all_data_sorted['TIEFEMD'] = all_data_sorted['TIEFEMD'].round(decimals=2)
    all_data_sorted['DEPTHFROM'] = all_data_sorted['DEPTHFROM'].round(decimals=2)
    all_data_sorted['DEPTHTO'] = all_data_sorted['DEPTHTO'].round(decimals=2)

# Export raw data to file

In [27]:
def export_raw_data():
    # Export all data to csv

    bh_raw_file = '_'.join(['bh_raw', str(now)]) +'.csv'
    bh_raw_path = os.path.join(out_dir_run, bh_raw_file)
    all_data_sorted.to_csv(bh_raw_path,index=None, sep=';')

<font color=red><h1>Run script</h1></font>

In [14]:
# Call function "make_run_dir()"
make_run_dir(out_dir_run)

Output directory not existing, directory will de created


In [15]:
# Call function "read_input()"
read_input(in_dir_bh)

In [16]:
# Call function "check_dtypes)()"
check_dtypes()

Data types of selected data:
 XCOORD        float64
YCOORD        float64
ZCOORDB       float64
ORIGNAME       object
NAMEPUB        object
SHORTNAME      object
BOHREDAT       object
GRUND          object
RESTRICTIO     object
TIEFEMD       float64
DEPTHFROM     float64
DEPTHTO       float64
LAYERDESC      object
dtype: object


In [17]:
# Call function "correct_dates()"
correct_dates()

Max:  <class 'str'> 9999-09-09 00:00:00
Min:  <class 'str'> 01.01.1111
Max:  <class 'float'> nan
Min:  <class 'float'> nan


In [18]:
# call function "create_stats()"
create_stats()

Number of individual borehole:  6060
Number of individual layers:  85822
count    6060.000000
mean       14.159076
std        17.686057
min         1.000000
25%         7.000000
50%        11.000000
75%        16.000000
max       400.000000
Name: DEPTHFROM, dtype: float64


In [19]:
correct_dates()

Max:  <class 'pandas._libs.tslibs.timestamps.Timestamp'> 2200-01-01T00:00:00.000000000
Min:  <class 'pandas._libs.tslibs.timestamps.Timestamp'> 1700-01-01T00:00:00.000000000


In [24]:
sort_data()

Shape of data set:  (85822, 106)


In [25]:
round_data()

In [28]:
export_raw_data()

<font color=green><h1>^^^^^^^^^^^^^^^ bis hier</h1></font>

# Preprocessing PRIVATE data

### Catch all columns and column numbers in dict
This shall help to easily see which cloumn is mentioned in warnings

In [ ]:
#10,13,17,18,20,71,100

In [ ]:
# copy all columns name to list "cols"
cols = all_data_sorted.columns.to_list()

# Create a dictionry of column names and locations
col_dicts = {}

keys = len(cols)
values = cols

for i in range(keys):
        col_dicts[i] = values[i]

print(col_dicts)


In [ ]:
# Check the shape (rows, columns) of the combined files
all_data_sorted.shape

### Checking for duplicated rows  and eliminate them

In [ ]:
# Create a list of columns except the index column, in order to use it as a subset for finding duplicate rows
cols_2 = cols[0:-1]

In [ ]:
# Find duplicate rows (checking duplicates based on all columns)
dup_rows = all_data_sorted[all_data_sorted.duplicated(subset=cols_2)]
print("Duplicate Rows except first occurrence based on all columns are :", dup_rows['LONGNAME'].count())

#Show duplicate entries if exisiting
if len(dup_rows) > 0:
    dup_rows.head()
else:
    pass

In [ ]:
# Drop all duplicated rows and save result to "all_data_unique"
if len(dup_rows) > 0:
    all_data_unique = all_data_sorted.drop_duplicates(subset=cols_2)
    print(len(dup_rows), " dropped")
    print("shape all_data_unipue: ", all_data_unique.shape)
    print("shape all_data_sorted: ", all_data_sorted.shape)
else:
    all_data_unique = all_data_sorted
    print("No rows dropped")
    #print("shape all_data_unipue: ", all_data_unique.shape)
    print("shape all_data_sorted: ", all_data_sorted.shape)

In [ ]:
# Select only relevant columns and save into "all_data_unique_sel"
all_data_unique_sel = all_data_unique[sel_cols]

# For entire data set with all columns: Save all into "all_data_unique_sel"
#all_data_unique_sel = all_data_unique


In [ ]:
all_data_unique_sel.head(5)

# <font color='red'> > Create GDF for all data PRIVATE data</font>

In [ ]:
# Convert dataframe "all_data_unique_sel" to geodataframe "gdf" and set inital crs to epsg:2056 
geom = [Point(xy) for xy in zip(all_data_unique_sel.XCOORD, all_data_unique_sel.YCOORD)]
crs = {"init" : "epsg:2056"}
private_bh = gpd.GeoDataFrame(all_data_unique_sel, crs=crs, geometry=geom)

In [ ]:
# Check gdf
private_bh.head()
#private_bh.shape

# Plot GDF

In [ ]:
# Load swiss boundary from shapefile
ch_perimeter = gpd.read_file('./data/input/helper/swissBOUNDARIES3D_1_3_TLM_LANDESGEBIET.shp')

In [ ]:
# Select only swiss perimeter
ch_perimeter = ch_perimeter[ch_perimeter['ICC'] == 'CH']
ch_perimeter

In [ ]:
# Create buffer around CH-Perimeter
ch_peri_buf_10km = ch_perimeter.buffer(20000) # 20km

In [ ]:
# Plot GeoDataFrame together with swiss boundary 
base = ch_perimeter.plot(color='white', edgecolor='black')
lyr_1 = ch_peri_buf_10km.plot(ax=base, edgecolor='blue', facecolor='none')
private_bh.plot(ax=lyr_1, color='red', markersize=5);

### CLIP GDF to swiss boundary

In [ ]:
# Clip the data using GeoPandas clip
gdf_clip = gpd.clip(gdf, ch_perimeter)

### Reproject EPSG:2056 to EPSG:4326

In [ ]:
#Write re-projected coordinates into new column
#gdf_trans = gdf
private_bh["geom4326"] = private_bh["geometry"].to_crs("epsg:4326")

private_bh["x4326"] = private_bh["geom4326"].apply(lambda p: p.x)
private_bh["y4326"] = private_bh["geom4326"].apply(lambda p: p.y)

private_bh.shape

In [ ]:
private_bh.head()

In [ ]:
private_bh.shape

In [ ]:
# Rename columns according to conventions
private_bh.columns = export_pri_cols
private_bh.head()

### Use private_bh as basis for creating OPEN data set and 2D data set

# <font color='red'> Export PRIVATE data to file >>>>> </font>

In [ ]:
# Export PRIVATE data to csv
#all_data_sorted.to_csv("./data/output/Schichtdaten/bh_raw_201109.csv",index=None)
bh_private_file = '_'.join(['bh_private', str(now), str(run_num).zfill(2),]) +'.csv'
bh_private_path = os.path.join(out_dir_run,bh_private_file)
private_bh.to_csv(bh_private_path,index=None, sep=';')

# Preprocessing data for OPEN dataset
### Introduce columns "start" and "end" to to store start and end depth of entire borehole to each row 

In [ ]:
# Reset columns names to original
private_bh.columns = cols_pub
private_bh.head()

In [ ]:
private_bh.shape

In [ ]:
# create new columns "start" and "end" and fill it with start and end depth
private_bh['start'] = private_bh.groupby('SHORTNAME')['DEPTHFROM'].transform('min')
private_bh['end'] = private_bh.groupby('SHORTNAME')['DEPTHTO'].transform('max')

# show head
private_bh.sort_values(by=["SHORTNAME", "DEPTHFROM"]).head(10)


### Split layer based on restriction level
Restriction level:
* g = restricted
* b = restricted until date specified
* f = free, not restricted

In [ ]:
# Select row which have "RESTRICTIO" == "g"
# Select row which have "RESTRICTIO" == "f"
# Select row which have "RESTRICTIO" == "b"

df_g = private_bh.loc[private_bh["RESTRICTIO"] == "g"].sort_values(by=["SHORTNAME", "DEPTHFROM"])
df_f = private_bh.loc[private_bh["RESTRICTIO"] == "f"].sort_values(by=["SHORTNAME", "DEPTHFROM"])
df_b = private_bh.loc[private_bh["RESTRICTIO"] == "b"].sort_values(by=["SHORTNAME", "DEPTHFROM"])

# Sum of all rows from df_g, df_b, df_f
df_sum = df_g.shape[0] + df_b.shape[0] + df_f.shape[0]

print("All unique data: ",private_bh.shape)
print("Restricted unique data: ", df_g.shape)
print("Restricted until unique data: ", df_b.shape)
print("Non-restricted unique data: ", df_f.shape)
print("Restricted + restircted until + Non-restricted data: ", df_sum)
print("Difference between all unique data and restriceted + Non-restricted: ", private_bh.shape[0] - df_sum)


In [ ]:
# delete duplicates in df_g
df_g_unique = private_bh.loc[private_bh["RESTRICTIO"] == "g"].sort_values(by=["SHORTNAME", "DEPTHFROM"]).drop_duplicates("SHORTNAME")
print('df_g_unique: ', df_g_unique.shape)

# delete duplicates in df_b
df_b_unique = private_bh.loc[private_bh["RESTRICTIO"] == "b"].sort_values(by=["SHORTNAME", "DEPTHFROM"]).drop_duplicates("SHORTNAME")
print('df_b_unique: ', df_b_unique.shape)


In [ ]:
df_g_unique.head()

In [ ]:
# Append df_b to df_g, so that all restriced data are in one dataframe
df_g_unique = df_g_unique.append(df_b_unique)

In [ ]:
df_g_unique.shape

### Replace layer details with undefined values

In [ ]:
# Overwrite "DEPTHFROM" and "DEPTHTO" with "start" and "end", respectively
df_g_unique["DEPTHFROM"] = df_g_unique["start"]
df_g_unique["DEPTHTO"] = df_g_unique["end"]

# Overwrite "LAYERDESC" of restricted boreholes ("RESTRICTIO = "g) with "Undefined"
df_g_unique[["LAYERDESC", "ORIGGEOL", "ORIGNAME", "BOHRTYP", "CHRONOSTR", "LITHOLOGY", "LITHOSTRAT"]] = "Undefined"

### Combine free and restricted data into one

In [ ]:
# Combine the two dataframe df_g and df_f
frames = [df_g_unique, df_f]
open_bh = pd.concat(frames)

# Drop the columns "start" and "end"
open_bh = open_bh.drop(["start", "end"], axis=1)


In [ ]:
# Sort dataset by unique-ID "SHORTNAME" and "DEPTHFROM"
open_bh = open_bh.sort_values(by=["SHORTNAME", "DEPTHFROM"])

In [ ]:
open_bh.shape

In [ ]:
open_bh['SHORTNAME'].nunique()

In [ ]:
open_bh.columns = export_pub_cols
open_bh.head()

# <font color='red'> Export OPEN data </font>

In [ ]:
# Export PRIVATE data to csv
#all_data_sorted.to_csv("./data/output/Schichtdaten/bh_raw_201109.csv",index=None)
bh_open_file = '_'.join(['bh_open', str(now), str(run_num).zfill(2),]) +'.csv'
bh_open_path = os.path.join(out_dir_run,bh_open_file)
open_bh.to_csv(bh_open_path,index=None, sep=';')

# Prepocessing 2D Data
### Create 2D data set by aggregating records by "SHORTNAME", each record becomes one borehole

In [ ]:
# Reset columns names to orginal
open_bh.columns = cols_pub
open_bh.head()

In [ ]:
# Drop all duplicated record based on unique-ID "SHORTNAME" apart from first record

bh_2d = open_bh.drop_duplicates(subset=["SHORTNAME"], keep="first")

In [ ]:
bh_2d.shape

In [ ]:
bh_2d.head()

### Select only relevant columns for 2D dataset

In [ ]:
# Select only relvant columns and save it back into "df_all_unique"
#df_all_unique = df_all_unique[["index","XCOORD","YCOORD","ZCOORDB","ORIGNAME","NAMEPUB","SHORTNAME", 'BOHREDAT',"BOHRTYP", 'GRUND',"RESTRICTIO","TIEFEMD"]]
bh_2d = bh_2d[cols_2D]


### Create new Atribute for Link to swissgeol

In [ ]:
#Define parameter 
baseURL = 'https://swissgeol.ch/?'
para_sep = '&'

layer_key = 'layers='
layer_value = 'boreholes'

layer_vis = 'layers_visibility='
layer_vis_value = 'true'

layer_trans = 'layers_transparency='
layer_trans_value = '0'
link_key = 'zoom_to='
link_sep = ','

# create Link
#df_all_unique['link'] = baseURL + layer_key + layer_value + para_sep + layer_vis + layer_vis_value + para_sep + layer_trans + layer_trans_value + para_sep + link_key + df_all_unique['x4326'].map(str) + link_sep + df_all_unique['y4326'].map(str) +  link_sep + df_all_unique['ZCOORDB'].map(str)
bh_2d['link'] = baseURL + layer_key + layer_value + para_sep + layer_vis + layer_vis_value + para_sep + layer_trans + layer_trans_value + para_sep + link_key + bh_2d['x4326'].map(str) + link_sep + bh_2d['y4326'].map(str) +  link_sep + '0'

bh_2d.head(5)



In [ ]:
print(bh_2d['link'][0])

In [ ]:
#sort how many unique record are in the dataset

bh_2d = bh_2d.sort_values(by=["SHORTNAME"])
bh_2d.shape

# <font color='red'> Export 2D data </font>

In [ ]:
# Export PRIVATE data to csv

bh_2d_file = '_'.join(['bh_2D', str(now), str(run_num).zfill(2),]) +'.csv'
bh_2d_path = os.path.join(out_dir_run,bh_2d_file)
bh_2d.to_csv(bh_2d_path,index=None)